In [1]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [2]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present (though your files use decimals).
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)


In [3]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

Phương pháp Viền quanh tìm ma trận nghịch đảo

## Thuật toán

In [4]:
def compute_theta(A: np.ndarray, B_prev: np.ndarray, k: int) -> float:
    row = A[k-1, :k-1]
    col = A[:k-1, k-1]
    a_kk = A[k-1, k-1]
    theta = float(row @ B_prev @ col - a_kk)
    if np.isclose(theta, 0):
        raise ZeroDivisionError(f"θ_{k} is zero; block singular at size {k}.")
    return theta

In [5]:
def compute_b_nn(A: np.ndarray, B_prev: np.ndarray, k: int) -> float:
    # b_{k,k} = -1/θ_k
    theta = compute_theta(A, B_prev, k)
    return -1.0 / theta

In [6]:
def compute_beta_col(A: np.ndarray, B_prev: np.ndarray, k: int) -> np.ndarray:
    # β_{1,k-1}: last column block (excluding b_{k,k})
    alpha_col = A[:k-1, k-1]
    theta = compute_theta(A, B_prev, k)
    return B_prev.dot(alpha_col) / theta

In [7]:
def compute_beta_row(A: np.ndarray, B_prev: np.ndarray, k: int) -> np.ndarray:
    # β_{k-1,1}: last row block (excluding b_{k,k})
    alpha_row = A[k-1, :k-1]
    theta = compute_theta(A, B_prev, k)
    return alpha_row.dot(B_prev) / theta

In [8]:
def update_top_left_block(A: np.ndarray, B_prev: np.ndarray, k: int, beta_row: np.ndarray) -> np.ndarray:
    # B_new = B_prev @ (I - α_col ⊗ β_row)
    alpha_col = A[:k-1, k-1]
    I = np.eye(k-1)
    return B_prev.dot(I - np.outer(alpha_col, beta_row))

In [9]:
def block_matrix_recursion(A: np.ndarray) -> np.ndarray:
    """
    Computes A^{-1} via block recursive inversion on leading principal minors.
    """
    n = A.shape[0]
    # Base case k=1
    theta1 = A[0,0]
    if np.isclose(theta1, 0):
        raise ZeroDivisionError("θ_1 is zero; A[0,0] is singular.")
    B_prev = np.array([[1.0/theta1]])
    # Recursively build inverse for k=2..n
    for k in range(2, n+1):
        print(f"Theta {k}:", compute_theta(A, B_prev, k))
        # Compute new blocks
        beta_col = compute_beta_col(A, B_prev, k)
        beta_row = compute_beta_row(A, B_prev, k)
        b_kk = compute_b_nn(A, B_prev, k)
        # Update top-left (k-1)x(k-1)
        B_tl = update_top_left_block(A, B_prev, k, beta_row)
        # Assemble B_k
        Bk = np.zeros((k, k))
        Bk[:k-1, :k-1] = B_tl
        Bk[:k-1, k-1] = beta_col
        Bk[k-1, :k-1] = beta_row
        Bk[k-1, k-1] = b_kk
        B_prev = Bk
        print(f"Size {k}:\n", B_prev, "\n")
    return B_prev

In [10]:
def inverse_via_ata(A: np.ndarray) -> np.ndarray:
    """
    Computes A^{-1} indirectly by inverting M = A^T A via block recursion,
    then A^{-1} = M^{-1} A^T.
    """
    M = A.T.dot(A)
    M_inv = block_matrix_recursion(M)
    return M_inv.dot(A.T)

## Kết quả

In [11]:
#Original matrix Ax=B
A = input_matrix('BLMT_input_A.txt', convert_fractions=False)

print("\nMatrix A:"); output_matrix(A)
print("\nMatrix M = A^T * A:"); output_matrix(A.T.dot(A))


Matrix A:
 10.0 -2.0 -5.0 -7.0 -9.0 -9.0
-10.0 -5.0 -7.0  7.0 -5.0  8.0
  6.0  6.0 -8.0  3.0 -8.0  9.0
  7.0 -1.0  8.0 -3.0 -7.0  0.0
  8.0  9.0  2.0  0.0 -5.0  0.0
 -9.0 -7.0  1.0 -2.0 -2.0 -3.0

Matrix M = A^T * A:
 430.0  194.0   35.0 -125.0 -159.0  -89.0
 194.0  196.0    0.0   14.0  -29.0   53.0
  35.0    0.0  207.0  -64.0   76.0  -86.0
-125.0   14.0  -64.0  120.0   29.0  152.0
-159.0  -29.0   76.0   29.0  248.0  -25.0
 -89.0   53.0  -86.0  152.0  -25.0  235.0


In [12]:
A_inv = inverse_via_ata(A)
print("Inverse of A result:\n", A_inv)

Theta 2: -108.47441860465116
Size 2:
 [[ 0.00420204 -0.00415916]
 [-0.00415916  0.00921876]] 

Theta 3: -201.85249978561015
Size 3:
 [[ 0.0043092  -0.00426523 -0.00072861]
 [-0.00426523  0.00932375  0.00072117]
 [-0.00072861  0.00072117  0.00495411]] 

Theta 4: -28.571058228347155
Size 4:
 [[ 0.01496363 -0.01619028  0.00344043  0.01931087]
 [-0.01619028  0.02267096 -0.00394506 -0.02161385]
 [ 0.00344043 -0.00394506  0.00658544  0.00755627]
 [ 0.01931087 -0.02161385  0.00755627  0.03500045]] 

Theta 5: -106.65300179638507
Size 5:
 [[ 0.02606694 -0.02629327  0.00051205  0.02802791  0.01020328]
 [-0.02629327  0.03186375 -0.0012805  -0.02954556 -0.00928404]
 [ 0.00051205 -0.0012805   0.00735777  0.00525725 -0.00269101]
 [ 0.02802791 -0.02954556  0.00525725  0.04184406  0.00801044]
 [ 0.01020328 -0.00928404 -0.00269101  0.00801044  0.0093762 ]] 

Theta 6: -18.66271256916687
Size 6:
 [[ 0.02935248 -0.02788308  0.0021065   0.04988769  0.00774149 -0.01326833]
 [-0.02788308  0.03263303 -0.00205